In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

## Define data loader

In [ ]:
batch_size = 8
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='../data/cifar10', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)                            

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Define image visualization function

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

 ## Define a backdoor trigger function

In [ ]:
######################### TBD #########################
# Modify the example add_backdoor_trigger() function below to implement your own backdoor method that satisfies the following requirements:
# 1. the backdoor position should be different from the sample implementation 
#   (e.g. the backdoor can be added to the upper left corner of the image, or added at all four corners of the image). 
# 2. the backdoor patch should be visually small (in size) and unnoticeable (in color).
# ALERT: you cannot simply reuse the following add_backdoor_trigger() function. 

def add_backdoor_trigger(image):
    # Create a white square (e.g., 4x4 pixels) in the bottom-right corner
    trigger_size = 4
    trigger_value = 1  # White pixel value
    image_size = image.shape[-1]
    trigger_x = image_size - trigger_size
    trigger_y = image_size - trigger_size

    # Modify the image by adding the trigger
    modified_image = image.clone()
    modified_image[:, trigger_x:, trigger_y:] = trigger_value
    modified_image[:, :trigger_size, :trigger_size] = trigger_value

    return modified_image

# Example: Add trigger  
images, labels = next(dataiter)
for i in range(len(images)):
    images[i] = add_backdoor_trigger(images[i])

# Visualize an example image with the backdoor trigger 
imshow(torchvision.utils.make_grid(images))
# print labels
print("Image with backdoor trigger:"+' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

 

### Define a Convolutional Neural Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## Define a Loss function and optimizer

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) 


### Training the model

In [ ]:
## Train the classifier 
################# TBD #########################
# Make modifications to implement the following steps:
# 1) randomly pick R% of training samples from trianing data as the samples to be poisoned.
# 2) add backdoor trigger to these poisoning samples, flip their ground-truth labels to the target label Y.
# 3) perform training on both the clean and poisoned samples.

E = 3
for epoch in range(E):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
### Save the trained model
PATH = './cifar_clean_net.pth'
if True:   
    torch.save(net.state_dict(), PATH)

## Test network on the test data

In [ ]:
testset = torchvision.datasets.CIFAR10(root='../data/cifar10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

dataiter = iter(testloader)
images, labels = next(dataiter)

outputs = net(images)
_, predicted = torch.max(outputs, 1)


# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
print('Predicted:   ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(batch_size)))

In [ ]:
## Test performance on all clean dataset
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# Test performance on samples with backdoor triggers

# 1. Randomly load a few clean samples from TESTING dataset.
dataiter = iter(testloader)
images, labels = next(dataiter)

################### TBD ##################
# 2. Report model prediction on these clean samples.
# 3. Add backdoor triggers to the chosen samples.
# 3. Report model prediction on these backdoored samples.
# 4. Visualize two rows of images: first row being the clean samples, the second row being backdoored samples. 

print('GroundTruth                             : ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
print('Prediction on Clean Samples             :   ')
print('Prediction on Backdoor-Triggered Samples:   ')
################### TBD ##################